# yfinance

**yfinance** is a library for accessing financial data from Yahoo Finance.

https://github.com/ranaroussi/yfinance

**yfinance documentation**: https://ranaroussi.github.io/yfinance/index.html


# Import Required Libraries

In [ ]:
# Yahoo Finance API
import yfinance as yf


'yfinance': Used to fetch stock data from Yahoo Finance [Source](https://pypi.org/project/yfinance/)

```python